In [1]:
import pandas as pd

In [9]:
gambling_keywords_df = pd.read_csv("SlotSchool3NF_Tables/Gambling_Keywords.csv")
counties_df = pd.read_csv("SlotSchool3NF_Tables/Counties.csv")
gamblingplace_keywords_df = pd.read_csv("SlotSchool3NF_Tables/GamblingPlace_Keywords.csv")
gambling_places_df = pd.read_csv("SlotSchool3NF_Tables/Gambling_Places.csv")
addresses_df = pd.read_csv("SlotSchool3NF_Tables/Addresses.csv")
schools_df = pd.read_csv("SlotSchool3NF_Tables/Schools.csv")
school_place_proximity_df = pd.read_csv("SlotSchool3NF_Tables/School_Place_Proximity.csv")

In [ ]:
# rename school name coordinates to easily distinguish between the original name/address coordinates orginially in the schools csv
# schools_df.rename(columns={"SchoolLatitude": "SchoolNameLatitude"}, inplace=True)
# schools_df.rename(columns={"SchoolLongitude": "SchoolNameLongitude"}, inplace=True)

In [3]:
# remove trusted_location from GamblingPlaces
gambling_places_df = gambling_places_df.drop("TrustedLocation", axis=1)

In [4]:
gambling_places_df.to_csv("SlotSchool3NF_Tables/Gambling_Places.csv", index=False)

In [10]:
# dropped columns to satisfy BCNF
schools_df = schools_df.drop(columns=["SchoolNameLatitude", "SchoolNameLongitude", "CoordDistance"])

In [11]:
schools_df.to_csv("SlotSchool3NF_Tables/Schools.csv", index=False)

In [4]:
# Define suspected primary keys for each table and check if they are unique for BCNF
key_checks = {
    "gambling_keywords_df": gambling_keywords_df["KeywordId"].is_unique,
    "counties_df": counties_df["CountyId"].is_unique,
    "gamblingplace_keywords_df": (gamblingplace_keywords_df[["GamblingPlaceId", "KeywordId"]]
                                   .duplicated().sum() == 0),
    "gambling_places_df": gambling_places_df["GamblingPlaceId"].is_unique,
    "addresses_df": addresses_df["SchoolAddressId"].is_unique,
    "schools_df": schools_df["SchoolId"].is_unique,
    "school_place_proximity_df": (school_place_proximity_df[["SchoolId", "GamblingPlaceId"]]
                                   .duplicated().sum() == 0)
}

key_checks

{'gambling_keywords_df': True,
 'counties_df': True,
 'gamblingplace_keywords_df': np.True_,
 'gambling_places_df': True,
 'addresses_df': True,
 'schools_df': False,
 'school_place_proximity_df': np.True_}